<a href="https://colab.research.google.com/github/Aditya-y9/WhyT/blob/main/QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers
!pip install FPDF

  Preparing metadata (setup.py) ... done
  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=be4fa3d51ae4139e069178660a9c38907bedb72cb30ad46e012ebb006eedcbef
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built FPDF


In [9]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 19.0 MB/s eta 0:00:00


In [10]:
import fitz
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
    return text

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_subtopics(text, model_name="gpt2", max_length=200):
    # Load pre-trained GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Tokenize the input text
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate subtopics using the model
    output = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

    # Decode the generated output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

def write_back_to_pdf(generated_text, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(generated_text)

# Example usage
pdf_path = "your_document.pdf"
output_path = "formatted_output.pdf"

text = extract_text_from_pdf(r"/content/transcript_output.pdf")
formatted_subtopics = generate_subtopics(text)
write_back_to_pdf(formatted_subtopics, output_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 1024, but `max_length` 

RuntimeError: The expanded size of the tensor (200) must match the existing size (1025) at non-singleton dimension 0.  Target sizes: [200].  Tensor sizes: [1025]

In [14]:
from transformers import pipeline

# Load the question-answering pipeline with the deepset/roberta-base-squad2 model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")

def ask_question(context, question):
    # Use the question-answering model to get an answer
    result = qa_pipeline(context=context, question=question, max_length=512)  # Increase max_length
    answer = result['answer']

    return answer

# Example usage
text = extract_text_from_pdf(r"/content/transcript_output.pdf")
text
question = input("Enter your question: ")

answer = ask_question(text, question)
print("Answer:", answer)


Enter your question: when will surface get covered in volcanoes?
Answer: 10 000 years into the future
